# 机器学习基石 - hw1 - Q15~17

前期了解知识点：
* numpy 和 pandas 的关系（[参考 1](https://www.douban.com/note/635632989/)）

## 零、构思

- [x] 从文件读取数据
- [x] 数据转化为 x, y, w 的形式
- [x] 编写 PLA 单次修正的函数
- [x] 设定初始值
- [x] 创建需要最终记录的变量
- [x] 执行 PLA 训练，输出所需结果

本程序实验环境为：

* python 3.7 (用的 conda 环境)
* numpy 1.17
* pandas 0.25

## 一、数据集读取 - pandas


### 1.1 从文件读取数据集


> [pandas | How do I read and write tabular data?](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/02_read_write.html#min-tut-02-read-write)

原始作业数据为 `.dat` 文件，查看发现该文件 `空格` 和 `Tab` 混用了，所以需要手动设置分隔符。

[用 pandas 进行数据读取](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html#pandas.read_csv)：pandas 中，
* [`read_table`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_table.html#pandas.read_table) 函数默认分隔符为 `\t`
* 而 [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html#pandas.read_csv) 函数默认分隔符为 `,`

两者之间只有默认分隔符的区别。推荐统一使用 `read_csv`，分隔符不同时进行手动指定。（参见 [github 上关于不推荐用 read_table 的讨论](https://github.com/pandas-dev/pandas/issues/25220)）

In [1]:
import pandas as pd
import numpy as np

# 读取 .dat 文件，names 指定列名，delim_whitespace 设为 True 等同于设置 sep='\s+'
dataset = pd.read_csv("resource\hw1_15_train.dat", names=['x1', 'x2', 'x3', 'x4', 'y'], delim_whitespace=True)

# 查看前指定行数据（默认为 5），检查是否正确读取 / dataset.tail(n) 查看末几行
dataset.head()

,x1,x2,x3,x4,y
0,0.97681,0.107230,0.64385,0.29556,1
1,0.67194,0.241800,0.83075,0.42741,1
2,0.20619,0.233210,0.81004,0.98691,1
3,0.51583,0.055814,0.92274,0.75797,1
4,0.70893,0.108360,0.33951,0.77058,1


In [2]:
# 查看指定列
# dataset['x1']

# 查看每列数据类型（dtypes 是 DataFrame 和 Series 的一个属性，而不是一个函数）
# dataset.dtypes

# 查看描述性统计数据
# dataset.describe()

# 查看 technical information（其中 entries 指“条目”，即有多少行）
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
x1    400 non-null float64
x2    400 non-null float64
x3    400 non-null float64
x4    400 non-null float64
y     400 non-null int64
dtypes: float64(4), int64(1)
memory usage: 15.8 KB


In [3]:
type(dataset)

pandas.core.frame.DataFrame

`pd.read_csv` 返回的数据类型为 `DataFrame`.

pandas 有两个主要的数据类型：`DataFrame` 和 `Series`.
* `DataFrame` 相当于一个二维表格
* `DataFrame` 的每一列是一个 `Series` 

### 1.2 数据集插入列

**将 x0 插入到数据表中**：[`DataFrame.insert`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.insert.html)

In [4]:
dataset.insert(0, 'x0', 1)  # 注意 x0 = 1 而不是 0 ……
dataset

,x0,x1,x2,x3,x4,y
0,1,0.976810,0.107230,0.643850,0.295560,1
1,1,0.671940,0.241800,0.830750,0.427410,1
2,1,0.206190,0.233210,0.810040,0.986910,1
3,1,0.515830,0.055814,0.922740,0.757970,1
4,1,0.708930,0.108360,0.339510,0.770580,1
...,...,...,...,...,...,...
395,1,0.712060,0.515690,0.181680,0.555700,1
396,1,0.175280,0.262500,0.830600,0.029669,-1
397,1,0.938950,0.939410,0.724960,0.956550,1
398,1,0.046136,0.944130,0.038311,0.268120,-1


## 二、数据处理 - numpy


### 2.1 用矩阵存储自变量与因变量


[`df.iloc`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html)

In [5]:
# 第一个 ':' 表示提取数据集的全部行，':-1' 表示提取除最后一列外的所有列
X = dataset.iloc[:, :-1].values

# todo: 怎么把 x0 加进去？
X

array([[1.      , 0.97681 , 0.10723 , 0.64385 , 0.29556 ],
       [1.      , 0.67194 , 0.2418  , 0.83075 , 0.42741 ],
       [1.      , 0.20619 , 0.23321 , 0.81004 , 0.98691 ],
       ...,
       [1.      , 0.93895 , 0.93941 , 0.72496 , 0.95655 ],
       [1.      , 0.046136, 0.94413 , 0.038311, 0.26812 ],
       [1.      , 0.072491, 0.2242  , 0.62592 , 0.67238 ]])

In [6]:
y = dataset.iloc[:, -1].values
y

array([ 1,  1,  1,  1,  1,  1, -1,  1, -1, -1,  1,  1,  1, -1, -1,  1,  1,
        1, -1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1, -1, -1,  1,  1, -1,
        1,  1, -1, -1,  1, -1, -1,  1, -1,  1,  1,  1, -1, -1,  1,  1,  1,
        1,  1,  1,  1,  1,  1, -1, -1,  1, -1,  1, -1, -1,  1, -1,  1, -1,
       -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1,
        1, -1,  1,  1,  1,  1,  1,  1,  1, -1,  1, -1,  1,  1, -1,  1,  1,
        1,  1, -1,  1,  1,  1,  1, -1,  1, -1,  1,  1, -1,  1,  1,  1,  1,
       -1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1, -1,
        1, -1,  1,  1,  1, -1,  1, -1, -1,  1, -1, -1,  1,  1,  1,  1, -1,
        1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1,  1, -1,  1, -1,  1, -1,
        1,  1, -1, -1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,
       -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1,
        1, -1,  1,  1, -1,  1, -1, -1,  1,  1,  1,  1,  1,  1,  1, -1,  1,
       -1,  1,  1,  1,  1

In [7]:
print("type(X):", type(X))
print("type(y):", type(y))

type(X): <class 'numpy.ndarray'>
type(y): <class 'numpy.ndarray'>


[numpy](https://numpy.org/devdocs/user/quickstart.html#the-basics) 最重要的数据类型就是 `ndarray`，即多维矩阵

In [8]:
# 查看 ndarray 的形状
X.shape

(400, 5)

## 三、PLA 算法

### 3.1 定义 sign 函数

numpy 本身有自带的 [`numpy.sign`](https://numpy.org/doc/stable/reference/generated/numpy.sign.html?highlight=sign#numpy.sign) 函数，不过题目中要求定义 sign(0) = -1，与标准的 sign 函数不太一样，所以自定义 sign 函数。

In [9]:
def sign(n):
    if n > 0:
        return 1
    else:
        return -1

### 3.2 定义 w 更新规则

In [10]:
def pla_update(wt, xt, yt):
    """根据规则更新 w"""
    w_updated = wt + yt * xt
    return w_updated

### 3.3 定义 PLA 执行过程

只要有修正，就需要重新检查所有的数据点，避免因后面某一次修正过度，导致之前的点为错误点。

In [11]:
def pla(w0, X, y, update_func):
    wt = w0.copy()  # 要用 copy
    t_up = 0  # 总修正次数
    t = 0  # 当前 w 已验证的正确点个数，只要有修正就重新计数
    
    i = 0
    while i < len(X):
        xt = X[i]
        yt = y[i]
        ht = sign(np.vdot(wt, xt))
        
        if ht != yt:  # 找到错误点
            wt = update_func(wt, xt, yt)
            t_up += 1
            t = 0
            # print(f"No.{t_up} update: {wt}")
        else:
            t += 1
            if t >= len(X):
                return t_up, wt
        
        # 循环查找数据中的点以确保每个点都正确
        i += 1
        if i >= len(X):
            i -= len(X)
    
    return None, None

## Q15

In [12]:
# 设定 w 初始值
w0 = np.zeros(5)

t_up, wg = pla(w0, X, y, pla_update)
print("------------------------------------------------------")
print("Number of Updates:", t_up)
print("Final wg:", wg)
print("------------------------------------------------------")

------------------------------------------------------
Number of Updates: 45
Final wg: [-3.         3.0841436 -1.583081   2.391305   4.5287635]
------------------------------------------------------


## Q16

* ndarray 如果直接用 `arr2 = arr1` 拷贝，则 `arr2 is arr1` 为 `True`，改变其中任一个，也会造成另一个的改变
* 为了仅拷贝值，使其是两个不同的对象，不会关联改变，需要用 `arr2 = arr1.copy()`

为打乱 X 和 y，同时不改变两者相对顺序，可以用 numpy 的 array indexing（参考：[StackOverflow -  Better way to shuffle two numpy arrays in unison](https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison))

In [13]:
def unison_shuffled_copies(arr1, arr2):
    """打乱两个 numpy array 的顺序而保持相对顺序不变"""
    assert len(arr1) == len(arr2)
    p = np.random.permutation(len(arr1))
    return arr1[p], arr2[p]

In [14]:
T_REPEAT = 2000

def pla_random(w0, X, y, update_func, t_repeat):
    sum_t_up = 0
    j = 0
    while j < t_repeat:
        X_rand, y_rand = unison_shuffled_copies(X, y)
        t_up_rand, _ = pla(w0, X_rand, y_rand, pla_update)
#         print(f"Exp.{j} t_up = {t_up_rand}")
        sum_t_up += t_up_rand
        j += 1
    avg_t_up = sum_t_up / t_repeat
    return avg_t_up

avg_t_up = pla_random(w0, X, y, pla_update, T_REPEAT)
print("------------------------------------------------------")
print(f"Average Number of Updates in {T_REPEAT} Experiments:", avg_t_up)
print("------------------------------------------------------")


------------------------------------------------------
Average Number of Updates in 2000 Experiments: 40.071
------------------------------------------------------


## Q17

根据题目要求，再次封装了之前的程序以便复用

In [15]:
def pla_update_with_factor(wt, xt, yt, yita=0.5):
    """根据规则更新 w"""
    w_updated = wt + yita * yt * xt
    return w_updated

avg_t_up2 = pla_random(w0, X, y, pla_update_with_factor, T_REPEAT)
print("------------------------------------------------------")
print(f"Average Number of Updates in {T_REPEAT} Experiments:", avg_t_up2)
print("------------------------------------------------------")


------------------------------------------------------
Average Number of Updates in 2000 Experiments: 40.0845
------------------------------------------------------
